In [8]:
from decouple import config
import pandas as pd
import numpy as np
import os
import glob as gb
import sys
import psycopg2
from sqlalchemy import create_engine

# Configura la conexión a PostgreSQL
user = config('DB_USER')
password = config('DB_PASSWORD')
host = config('DB_HOST')
database = config('DB_DATABASE')
table_name = 'cnv'
# Crea la conexión usando sqlalchemy
engine = create_engine(f'postgresql://{user}:{password}@{host}/{database}')


sys.path.insert(0, "../../")
from clases.bd.conexion2 import MyDatabase2
conn = MyDatabase2()

Conexion exitosa con la Base de datos:irvin_hisminsa


In [9]:
ruta_datos = os.path.abspath("../../../../../HISMINSA/CNV")
archivos_excel = gb.glob(os.path.join(ruta_datos, '*.xlsx'))

# Inicializar un DataFrame vacío
df = pd.DataFrame()

for archivo in archivos_excel:
    print("Leyendo archivo:", archivo)
    # Leer la primera hoja del archivo Excel
    df_cnv = pd.read_excel(archivo, sheet_name=0)
    
    # Concatenar al DataFrame principal
    df = pd.concat([df, df_cnv], ignore_index=True)

# Imprimir información sobre el DataFrame resultante
print("Información del DataFrame combinado:")
print(df.info())

# Ver las primeras filas del DataFrame combinado
print(df.head())







Leyendo archivo: d:\Irvin\HISMINSA\CNV\CNV_DOM_MADRE_CUSCO_081123.xlsx
Leyendo archivo: d:\Irvin\HISMINSA\CNV\CNV_LUGARNACIDO_CUSCO_081123.xlsx
Información del DataFrame combinado:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151103 entries, 0 to 151102
Data columns (total 53 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   NU_CNV                     151103 non-null  int64  
 1   Ubigeo_LugarNacido         151103 non-null  int64  
 2   DPTO_EESS                  151103 non-null  object 
 3   PROV_EESS                  151103 non-null  object 
 4   DIST_EESS                  151103 non-null  object 
 5   Ipress                     151103 non-null  int64  
 6   CO_LOCAL                   151103 non-null  int64  
 7   Nombre_EESS                151103 non-null  object 
 8   Diresa_Diris               151103 non-null  object 
 9   Institución                151103 non-null  object 
 10  Categoria          

In [11]:
df.columns = df.columns.str.lower()
df['fe_nacido'] = pd.to_datetime(df['fe_nacido'], format='%Y%m%d')
df['fe_crea'] = pd.to_datetime(df['fe_crea'], format='%Y%m%d%H%M%S')
df['nu_doc_madre'] = df['nu_doc_madre'].str.strip()
df_sin_duplicados = df.drop_duplicates(subset=['tipo_doc_madre','fe_nacido', 'nu_doc_madre'])
#df_sin_duplicados = df.drop_duplicates(subset=['nu_cnv','tipo_doc_madre','fe_nacido', 'nu_doc_madre'])

#df.head()

df_sin_duplicados.head(5)



,nu_cnv,ubigeo_lugarnacido,dpto_eess,prov_eess,dist_eess,ipress,co_local,nombre_eess,diresa_diris,institución,...,ubigeo_dom_madre,edad_madre,profesion_prof,colegiat_prof,tip_docum_prof,nu_doc_prof,prim_ape_prof,seg_ape_prof,prenombres_prof,fe_crea
0,91664719,80105,CUSCO,CUSCO,SAN SEBASTIAN,25210,25210,TUPAC AMARU,CUSCO,GOBIERNO REGIONAL,...,81003.0,24,2,25718.0,1,43851820.0,SERVAN,MONTOYA,LUZ ESTEPHANIE ...,2020-01-04 03:16:34
1,91664856,80101,CUSCO,CUSCO,CUSCO,2289,2289,HOSPITAL DE APOYO DEPARTAMENTAL CUSCO,CUSCO,GOBIERNO REGIONAL,...,80105.0,28,2,17847.0,1,40069101.0,VALENCIA,VILLACRESIS,NOEMI ...,2020-01-04 06:53:35
2,91664961,80601,CUSCO,CANCHIS,SICUANI,2384,2384,PAMPAPHALLA,CUSCO,GOBIERNO REGIONAL,...,80601.0,23,2,25868.0,1,41835843.0,ARAPA,APAZA,SILVIA ROXANA ...,2020-01-04 08:13:26
3,91661752,81205,CUSCO,QUISPICANCHI,CCATCA,2528,2528,CCATCCA,CUSCO,GOBIERNO REGIONAL,...,81205.0,24,2,27512.0,1,44862412.0,DUE?AS,POVEA,MELISSA ...,2020-01-02 08:52:48
4,91662087,80101,CUSCO,CUSCO,CUSCO,2289,2289,HOSPITAL DE APOYO DEPARTAMENTAL CUSCO,CUSCO,GOBIERNO REGIONAL,...,80407.0,30,2,17847.0,1,40069101.0,VALENCIA,VILLACRESIS,NOEMI ...,2020-01-02 12:06:14


In [11]:
# Inserta el DataFrame en la tabla de PostgreSQL


df_sin_duplicados.to_sql(table_name, engine,schema='maestros', if_exists='replace', index=False)

# Cierra la conexión
engine.dispose()

In [3]:
import pandas as pd
import numpy as np
import os
import glob as gb
import sys
from zipfile import ZipFile
from os import remove

import gzip
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from psycopg2.extras import execute_values
import psycopg2
host = '192.168.1.7'
user = 'admin_ic'
password = '213141'
database = 'irvin_hisminsa'
options="-c search_path=maestros"
port = 5432
pgconn = psycopg2.connect(
    host=host,
    user=user,
    password=password,
    database=database,
    port=port,
    options=options
)
# cursor
pgcursor = pgconn.cursor()
# codigo requerido
pgconn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)



pgcursor.execute("TRUNCATE TABLE maestros.nominal_trama_2024")
#ruta_datos = os.path.abspath("../../../data/2023/csv/")
# print(ruta_datos)

# carpeta contenedor de datos Zip
dx = gb.glob("D:/Irvin/Irvin/Python/data/2024/csv/*.csv")
print(dx)

# lista de df
list_df = []
for f in dx:
    ruta = f
    print(f)  

    #pgcursor.execute("TRUNCATE TABLE maestros.nominal_trama_2023")
    with open(f, mode='r', encoding="ISO-8859-1") as f:
        next(f)
        pgcursor.copy_from(f, 'nominal_trama',    sep=',', null='', columns=None)
    #   print("Carga Completada: "+name)
print("Termino el Proceso")

['D:/Irvin/Irvin/Python/data/2024/csv\\11_CUSCO_01.csv']
D:/Irvin/Irvin/Python/data/2024/csv\11_CUSCO_01.csv
Termino el Proceso
